In [ ]:
from simodin.Examples.tespy.power_plant.SimModel_powerplant import tespy_model
from simodin import interface as link

Initiate the SimModel class and the coresponding tespy network model:

In [2]:
my_model= tespy_model('powerplant')

In [3]:
my_model.init_model()
my_model.calculate_model()
my_model.technosphere

{'cooling water sink': technosphere_edge(edge_type=<QuantitativeEdgeTypes.technosphere: 'technosphere'>, source=<tespy_example.tespy_model object at 0x0000020C0CCACBC0>, target=None, comment=None, tags=None, properties=None, name='cooling water sink', amount=<Quantity(33222973.0, 'kilogram')>, uncertainty_type=None, loc=None, scale=None, shape=None, minimum=None, maximum=None, negative=None, description=None, dataset_correction=None, functional=False, model_unit=None, dataset_unit=None, allocationfactor=1.0, type=<technosphereTypes.output: 'output'>),
 'cooling water source': technosphere_edge(edge_type=<QuantitativeEdgeTypes.technosphere: 'technosphere'>, source=None, target=<tespy_example.tespy_model object at 0x0000020C0CCACBC0>, comment=None, tags=None, properties=None, name='cooling water source', amount=<Quantity(33222973.0, 'kilogram')>, uncertainty_type=None, loc=None, scale=None, shape=None, minimum=None, maximum=None, negative=None, description=None, dataset_correction=None, 

Create a new modelInterface class with the SimModel class as argument:

In [ ]:
my_interface= link.modelInterface('tespy powerplant',my_model)

In [5]:
my_interface.technosphere

{'cooling water sink': technosphere_edge(edge_type=<QuantitativeEdgeTypes.technosphere: 'technosphere'>, source=<tespy_example.tespy_model object at 0x0000020C0CCACBC0>, target=None, comment=None, tags=None, properties=None, name='cooling water sink', amount=<Quantity(33222973.0, 'kilogram')>, uncertainty_type=None, loc=None, scale=None, shape=None, minimum=None, maximum=None, negative=None, description=None, dataset_correction=None, functional=False, model_unit=None, dataset_unit=None, allocationfactor=1.0, type=<technosphereTypes.output: 'output'>),
 'cooling water source': technosphere_edge(edge_type=<QuantitativeEdgeTypes.technosphere: 'technosphere'>, source=None, target=<tespy_example.tespy_model object at 0x0000020C0CCACBC0>, comment=None, tags=None, properties=None, name='cooling water source', amount=<Quantity(33222973.0, 'kilogram')>, uncertainty_type=None, loc=None, scale=None, shape=None, minimum=None, maximum=None, negative=None, description=None, dataset_correction=None, 

Search for brightway25 datasets and asign them to the technosphere flows:

In [ ]:
import bw2data as bd

bd.projects.set_current('my_project')
my_interface.methods=[('ecoinvent-3.11',  'EF v3.1',  'climate change',  'global warming potential (GWP100)')]

ei=bd.Database('ecoinvent-3.11-cutoff')

In [7]:
ei_steam=[act for act in ei if 'heat production, at hard coal industrial furnace 1-10MW' in act['name']
 and 'Europe without Switzerland' in act['location']
  ][0]
ei_steam

'heat production, at hard coal industrial furnace 1-10MW' (megajoule, Europe without Switzerland, None)

In [8]:
my_interface.technosphere['heat source'].source=ei_steam

In [9]:
ei_water=[act for act in ei if 'market for tap water' in act['name']
 and 'Europe without Switzerland' in act['location']
  ][0]
ei_water

'market for tap water' (kilogram, Europe without Switzerland, None)

Set dataset correction value, for simple adjusting of the model results to the LCA datasets: 

In [10]:
my_interface.technosphere['cooling water source'].source=ei_water
my_interface.technosphere['cooling water source'].dataset_correction = 0.1 #blowdown rate

Calculate background impact:

In [11]:
my_interface.calculate_background_impact()

And the impact of the functional unit of the model. The functional unit is defined in the SimModel class. In this case it is the produced electricity, called "grid".

In [ ]:
my_interface.calculate_impact()

Model can be exported to a brightway dataset. If no database is passed as argument, a default database called "simulation_model_db" is created:

In [14]:
code= my_interface.export_to_bw()

In [ ]:
code

'tespy powerplant_grid_2025-07-28 20:45:41.640943'